In [1]:
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torch.optim import lr_scheduler

import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
import torchvision.models as torchmodels

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch.backends.cudnn as cudnn
cudnn.benchmark = True

In [2]:
def load_split_train_test(datadir, valid_size=.2, batch_size=32, num_workers=6, size=(200,200)):
    'Load train/test data'
    normalize = torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    train_transforms = transforms.Compose([transforms.Resize(size),
                                           transforms.ToTensor(),
                                           normalize,
                                       ])
    test_transforms = transforms.Compose([transforms.Resize(size),
                                          transforms.ToTensor(),
                                          normalize,
                                      ])
    train_data = datasets.ImageFolder(datadir,
                    transform=train_transforms)
    test_data = datasets.ImageFolder(datadir,
                    transform=test_transforms)
    num_train = len(train_data)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)
    from torch.utils.data.sampler import SubsetRandomSampler
    train_idx, test_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    train = torch.utils.data.DataLoader(train_data,
                   sampler=train_sampler, batch_size=batch_size, 
                                        num_workers=num_workers, pin_memory=True)
    test = torch.utils.data.DataLoader(test_data,
                   sampler=test_sampler, batch_size=batch_size, 
                                       num_workers=num_workers, pin_memory=True)
    return train, test

In [3]:
# Setup variables
batch_size = 128
epochs = 10
learning_rate = 1e-3
imgsize = (128, 128)
transfer = False
root = 'data/'
train_loader, test_loader = load_split_train_test(root, batch_size=batch_size, size=imgsize)

In [4]:
# Model
net = torchmodels.resnet50(pretrained=False)
ct = 0
if transfer:
    for child in net.children():
        ct += 1
        if ct < 7:
            for param in child.parameters():
                param.requires_grad = False
net.fc = nn.Linear(2048, 8)
# Other model-related shtuff
optimizer = optim.Adam(params=net.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()
exp_scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.5)
if torch.cuda.is_available():
    print("What an amazing world we live in...")
    net = net.cuda()
    criterion = criterion.cuda()
    
# Init weights
def init_weights(m):
        if type(m) == nn.Conv2d:
            torch.nn.init.kaiming_normal_(m.weight)
net.apply(init_weights)


What an amazing world we live in...


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [5]:
def train(net, train_loader, optimizer, scheduler, criterion, epoch, batch_size):
    'Train loop'
    net.train()
    scheduler.step()
    for batch_idx, (images, labels) in enumerate(train_loader):
        images, target = Variable(images), Variable(labels)
        if torch.cuda.is_available():
            images = images.cuda()
            target = target.cuda()
        optimizer.zero_grad()
        output = net(images)
        loss = criterion(output, target)

        loss.backward()
        optimizer.step()
        if (batch_idx + 1) % 20 == 0:
            batch_size = len(images)
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, (batch_idx + 1) * len(images), len(train_loader) * batch_size,
                100. * (batch_idx + 1) / len(train_loader), loss.item()))


def evaluate(model, data_loader, batch_size):
    'Evaluate loop'
    model.eval()
    loss = 0
    correct = 0
    with torch.no_grad():
        for _, (data, target) in enumerate(data_loader):
            data, target = Variable(data), Variable(target)
            if torch.cuda.is_available():
                data = data.cuda()
                target = target.cuda()

            output = model(data)

            loss += F.cross_entropy(output, target, size_average=False).data.item()

            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()


        loss /= len(data_loader.dataset)
        acc = float(100. * correct) / float(len(data_loader)*batch_size)
        print('Average loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)'.format(
            loss, correct, len(data_loader)*batch_size, acc))
    return acc

In [6]:
# Training step happens here
def lazy():
    for epoch in range(epochs):
        epoch += 1
        train(net, train_loader, optimizer, exp_scheduler, criterion, epoch, batch_size)
        #print('train accuracy: ')
        #tracc = evaluate(net, train_loader, batch_size)
        print('test_accuracy: ')
        teacc = evaluate(net, test_loader, batch_size)
lazy()

Train Epoch: 1 [2560/20352 (13%)]	Loss: 1.270259
Train Epoch: 1 [5120/20352 (25%)]	Loss: 1.370888
Train Epoch: 1 [7680/20352 (38%)]	Loss: 2.062752
Train Epoch: 1 [10240/20352 (50%)]	Loss: 1.251606
Train Epoch: 1 [12800/20352 (63%)]	Loss: 1.210032
Train Epoch: 1 [15360/20352 (75%)]	Loss: 1.221559
Train Epoch: 1 [17920/20352 (88%)]	Loss: 1.052256
test_accuracy: 


/home/teiv/anaconda3/lib/python3.7/site-packages/torch/nn/_reduction.py:46: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Average loss: 0.9496, Accuracy: 2553/5120 (49.863%)
Train Epoch: 2 [2560/20352 (13%)]	Loss: 1.131922
Train Epoch: 2 [5120/20352 (25%)]	Loss: 1.052589
Train Epoch: 2 [7680/20352 (38%)]	Loss: 0.984476
Train Epoch: 2 [10240/20352 (50%)]	Loss: 1.042102
Train Epoch: 2 [12800/20352 (63%)]	Loss: 1.122846
Train Epoch: 2 [15360/20352 (75%)]	Loss: 1.180307
Train Epoch: 2 [17920/20352 (88%)]	Loss: 1.167070
test_accuracy: 
Average loss: 0.2320, Accuracy: 2929/5120 (57.207%)
Train Epoch: 3 [2560/20352 (13%)]	Loss: 1.047372
Train Epoch: 3 [5120/20352 (25%)]	Loss: 1.083418
Train Epoch: 3 [7680/20352 (38%)]	Loss: 1.251954
Train Epoch: 3 [10240/20352 (50%)]	Loss: 1.065370
Train Epoch: 3 [12800/20352 (63%)]	Loss: 1.022250
Train Epoch: 3 [15360/20352 (75%)]	Loss: 1.168136
Train Epoch: 3 [17920/20352 (88%)]	Loss: 1.141750
test_accuracy: 
Average loss: 0.2199, Accuracy: 3065/5120 (59.863%)
Train Epoch: 4 [2560/20352 (13%)]	Loss: 0.967916
Train Epoch: 4 [5120/20352 (25%)]	Loss: 0.914249
Train Epoch: 4 [7680

In [7]:
# save model first iteration
torch.save(net.state_dict, './resnet50-round1')

In [8]:
# Lower learning rate
learning_rate = 4e-4
optimizer = optim.Adam(params=net.parameters(), lr=learning_rate, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()
exp_scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.5)

In [9]:
lazy()

Train Epoch: 1 [2560/20352 (13%)]	Loss: 0.641072
Train Epoch: 1 [5120/20352 (25%)]	Loss: 0.719089
Train Epoch: 1 [7680/20352 (38%)]	Loss: 0.503528
Train Epoch: 1 [10240/20352 (50%)]	Loss: 0.719566
Train Epoch: 1 [12800/20352 (63%)]	Loss: 0.613006
Train Epoch: 1 [15360/20352 (75%)]	Loss: 0.594290
Train Epoch: 1 [17920/20352 (88%)]	Loss: 0.806315
test_accuracy: 
Average loss: 0.2413, Accuracy: 3118/5120 (60.898%)
Train Epoch: 2 [2560/20352 (13%)]	Loss: 0.627106
Train Epoch: 2 [5120/20352 (25%)]	Loss: 0.606147
Train Epoch: 2 [7680/20352 (38%)]	Loss: 0.435087
Train Epoch: 2 [10240/20352 (50%)]	Loss: 0.508033
Train Epoch: 2 [12800/20352 (63%)]	Loss: 0.521972
Train Epoch: 2 [15360/20352 (75%)]	Loss: 0.451593
Train Epoch: 2 [17920/20352 (88%)]	Loss: 0.543754
test_accuracy: 
Average loss: 0.2380, Accuracy: 3220/5120 (62.891%)
Train Epoch: 3 [2560/20352 (13%)]	Loss: 0.281652
Train Epoch: 3 [5120/20352 (25%)]	Loss: 0.271877
Train Epoch: 3 [7680/20352 (38%)]	Loss: 0.215133
Train Epoch: 3 [10240/2